<a href="https://colab.research.google.com/github/vvivvi/kaggle-c1/blob/master/Kaggle_C1_text_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm as lgb
from sklearn.metrics import r2_score
import catboost
import gc

from catboost import CatBoostRegressor, Pool

import re
import os

for p in [np, pd, scipy, sklearn, lgb, catboost]:
    print (p.__name__, p.__version__)
    
DATA_FOLDER = 'competitive-data-science-predict-future-sales'
test_spec = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

index_cols=['item_id','shop_id','date_block_num']
date_block_val = 33
date_block_test = 35 # Dec 2015

test2submission_mapping_generated = False

numpy 1.18.1
pandas 0.25.3
scipy 1.4.1
sklearn 0.22.1
lightgbm 2.3.1
catboost 0.22


In [2]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [3]:
!pip install catboost

In [4]:
def write_predictions_by_array(array, filename):
  df=pd.DataFrame(array)
  df.columns=['item_cnt_month']
  df.to_csv(os.path.join(DATA_FOLDER, filename), index_label='ID')

In [5]:
def clipped_rmse(gt, predicted,clip_min=0, clip_max=20):
  target=np.minimum(np.maximum(gt,clip_min), clip_max)
  return np.sqrt((target-predicted)**2).mean()

In [6]:
# plan:
# list all feature files

# create data structure that tells in which feature file a given feature is found

# create evaluation function that evaluates a given combination
# of features and optionally writes test set predictions to a csv file
# the necessary feature files are loade automatically in memory

# first create a function for regressors implementing the sklearn fit/predict interface
# plan to use CatBoost for first experiments







In [7]:
feature_files = ['features_category.csv','features_lagged_basic.csv','features_targets.csv','features_text.csv']
feature_files += ['features_mean_encoded.csv']
feature_files += ['target_category_frequent_256_lagged.csv','target_category_frequent_256_within_shop_lagged.csv']
feature_files += ['target_category_frequent_32_lagged.csv']
feature_files += ['target_category_frequent_32_within_shop_lagged.csv']
feature_files += ['target_category_lagged.csv']
feature_files += ['target_category_tfidf_bigram_256_lagged.csv']
feature_files += ['target_category_tfidf_bigram_256_within_shop_lagged.csv']
feature_files += ['target_category_tfidf_bigram_32_lagged.csv']
feature_files += ['target_category_tfidf_bigram_32_within_shop_lagged.csv']
feature_files += ['target_category_tfidf_unigram_256_lagged.csv']
feature_files += ['target_category_tfidf_unigram_256_within_shop_lagged.csv']
feature_files += ['target_category_tfidf_unigram_32_lagged.csv']
feature_files += ['target_category_tfidf_unigram_32_within_shop_lagged.csv']
feature_files += ['target_category_within_shop_lagged.csv']
feature_files += ['target_item_lagged.csv']
# feature_files += ['target_item_within_shop_lagged.csv']
feature_files += ['target_lagged.csv']
feature_files += ['target_shop_lagged.csv']



In [8]:
feature2filename={}

# create data structure that tells in which feature file a given feature is found
for f in feature_files:
    df =pd.read_csv(DATA_FOLDER + '/' + f)
    for col in df.columns.values:
        feature2filename[col]=f
        
                    

In [9]:
# list all features
sorted(feature2filename.keys())

['Unnamed: 0',
 'date_block_num',
 'item_category_id',
 'item_category_id_enc_train',
 'item_category_id_enc_trainval',
 'item_id',
 'item_id_enc_train',
 'item_id_enc_trainval',
 'item_name_category_frequent_256',
 'item_name_category_frequent_256_enc_train',
 'item_name_category_frequent_256_enc_trainval',
 'item_name_category_frequent_32',
 'item_name_category_frequent_32_enc_train',
 'item_name_category_frequent_32_enc_trainval',
 'item_name_category_tfidf_bigram_256',
 'item_name_category_tfidf_bigram_256_enc_train',
 'item_name_category_tfidf_bigram_256_enc_trainval',
 'item_name_category_tfidf_bigram_32',
 'item_name_category_tfidf_bigram_32_enc_train',
 'item_name_category_tfidf_bigram_32_enc_trainval',
 'item_name_category_tfidf_unigram_256',
 'item_name_category_tfidf_unigram_256_enc_train',
 'item_name_category_tfidf_unigram_256_enc_trainval',
 'item_name_category_tfidf_unigram_32',
 'item_name_category_tfidf_unigram_32_enc_train',
 'item_name_category_tfidf_unigram_32_enc_t

In [31]:
def evaluate(model, feature_lists):
    
    # feature_lists : list of feature lists (= feature combinations) to be red into memory 
    # simultaneously and then evaluated in one go
    
    files_and_columns_to_read={}
    lagged=False
    
    flat_features = set([item for sublist in feature_lists for item in sublist])
    
    for feat in flat_features.union({'target'}):
        is_index=False
        for idx in index_cols:
            if feat == idx:
                is_index = True
        # index columns are present in every feature file, no need to read
        # them explicitly
        
        if is_index: 
            continue
            
        if re.search('lag', feat):
            lagged=True
        file = feature2filename[feat]
        featlist=files_and_columns_to_read.get(file,[])
        featlist += [feat]
        files_and_columns_to_read[file] = featlist
    print('file -> columns mapping: ', files_and_columns_to_read)   
    
    first=True
    for file in files_and_columns_to_read:
        to_read_cols = files_and_columns_to_read[file]
        to_keep_cols = list(set(to_read_cols + index_cols)) # add index columns, remove duplicates
        
        if first:
            all_data = pd.read_csv(DATA_FOLDER + '/' + file )[to_keep_cols]
            first=False
        else:
            df = pd.read_csv(DATA_FOLDER + '/' + file )[to_keep_cols]
            df = downcast_dtypes(df)
            all_data = pd.merge(all_data, df, on=index_cols)
            del df
            gc.collect()
        if lagged:
            all_data=all_data[all_data['date_block_num'] >= 14]
        all_data = downcast_dtypes(all_data)
        gc.collect()
        
        
     
    print('read columns:', all_data.columns.values)
    
    dates = all_data['date_block_num']
    
    y_train = np.clip(all_data.loc[dates <  date_block_val, 'target'].values,0,20)
    y_trainval = np.clip(all_data.loc[dates <  date_block_test, 'target'].values,0,20)
    y_val =  np.clip(all_data.loc[dates == date_block_val, 'target'].values,0,20)
    
    print(y_train.shape)
    
    lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':2
    }
              
    for features in feature_lists:    
        model = lgb.train(lgb_params, lgb.Dataset(all_data.loc[dates <  date_block_val, features], label=y_train), 100)
        pred_lgb_val = np.clip(model.predict(all_data.loc[dates ==  date_block_val, features]), 0, 20)
        #print('Validation R-squared for LightGBM is %f' % r2_score(y_val, pred_lgb_val))
        print('{}: Clipped RMSE {}'.format(features,clipped_rmse(y_val, pred_lgb_val)))
    
    generate_submission=False
    
    if generate_submission:
        
        shop_item2submissionid={}
        for idx, row in test_spec.iterrows():
            shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])] = row['ID']
    
        test_data=all_data.loc[dates == date_block_test, ['shop_id','item_id']]    
    
        testidx2submissionidx=np.zeros(test_data.shape[0], dtype=np.int32)
        for idx in range(test_data.shape[0]):
            row =test_data.iloc[idx]
            testidx2submissionidx[idx] = shop_item2submissionid[str(row['shop_id'])+'_'+str(row['item_id'])]
    
        print(len(testidx2submissionidx))    

#invert the mapping
        submissionidx2testidx=np.zeros(test_data.shape[0], dtype=np.int32)
        for i in range(test_data.shape[0]):
            submissionidx2testidx[testidx2submissionidx[i]]=i
    
        del test_data
        gc.collect()    
    

In [11]:
#evaluate(' ',['item_id_enc_train','shop_id','target_lag_2','target_lag_3','target_lag_4','target_lag_5','target_lag_12','target_lag_14'])

In [ ]:
# evaluate text features

In [20]:
text_categories = [c for c in sorted(feature2filename.keys()) if re.search('item_name_category',c)]
text_categories = [c for c in text_categories if not re.search('trainval',c)]

In [29]:
# validate text categories in one-by-one setting 
evaluate(' ',[[feat] for feat in text_categories])

file -> columns mapping:  {'features_mean_encoded.csv': ['item_name_category_tfidf_bigram_256_enc_train', 'item_name_category_frequent_32_enc_train', 'item_name_category_tfidf_unigram_256_enc_train', 'item_name_category_tfidf_unigram_32_enc_train', 'item_name_category_frequent_256_enc_train', 'item_name_category_tfidf_bigram_32_enc_train'], 'features_text.csv': ['item_name_category_tfidf_unigram_256', 'item_name_category_tfidf_bigram_32', 'item_name_category_frequent_32', 'item_name_category_tfidf_bigram_256', 'item_name_category_frequent_256', 'item_name_category_tfidf_unigram_32'], 'target_lagged.csv': ['target']}
read columns: ['item_name_category_tfidf_bigram_256_enc_train'
 'item_name_category_frequent_32_enc_train'
 'item_name_category_tfidf_unigram_256_enc_train' 'shop_id'
 'date_block_num' 'item_id'
 'item_name_category_tfidf_unigram_32_enc_train'
 'item_name_category_frequent_256_enc_train'
 'item_name_category_tfidf_bigram_32_enc_train'
 'item_name_category_tfidf_unigram_256'

In [38]:
lists = [['item_id','shop_id','item_category_id']]
lists += [['item_id_enc_train','shop_id','item_category_id']]
lists += [['item_id','shop_id','item_category_id_enc_train']]
lists += [['item_id_enc_train','shop_id_enc_train','item_category_id_enc_train']]
lists += [['item_id_enc_train','shop_id_enc_train','item_category_id_enc_train','item_id','shop_id','item_category_id']]
lists += [['item_category_id_enc_train','item_id','shop_id','item_category_id']]
evaluate(' ',lists)

file -> columns mapping:  {'features_mean_encoded.csv': ['item_category_id_enc_train', 'shop_id_enc_train', 'item_id_enc_train'], 'features_category.csv': ['item_category_id'], 'target_lagged.csv': ['target']}
read columns: ['shop_id' 'shop_id_enc_train' 'date_block_num' 'item_id'
 'item_category_id_enc_train' 'item_id_enc_train' 'item_category_id'
 'target']
(4499858,)
['item_id', 'shop_id', 'item_category_id']: Clipped RMSE 0.40559699177125225
['item_id_enc_train', 'shop_id', 'item_category_id']: Clipped RMSE 0.4222757449349657
['item_id', 'shop_id', 'item_category_id_enc_train']: Clipped RMSE 0.40809013618661927
['item_id_enc_train', 'shop_id_enc_train', 'item_category_id_enc_train']: Clipped RMSE 0.42615876960307214
['item_id_enc_train', 'shop_id_enc_train', 'item_category_id_enc_train', 'item_id', 'shop_id', 'item_category_id']: Clipped RMSE 0.42635583138890887
['item_category_id_enc_train', 'item_id', 'shop_id', 'item_category_id']: Clipped RMSE 0.40400694326213804


In [39]:
lists = [['item_category_id_enc_train','item_id','shop_id','item_category_id',feat] for feat in text_categories]
evaluate(' ',lists)


file -> columns mapping:  {'features_mean_encoded.csv': ['item_name_category_tfidf_bigram_256_enc_train', 'item_name_category_tfidf_unigram_32_enc_train', 'item_name_category_frequent_256_enc_train', 'item_category_id_enc_train', 'item_name_category_tfidf_bigram_32_enc_train', 'item_name_category_frequent_32_enc_train', 'item_name_category_tfidf_unigram_256_enc_train'], 'features_text.csv': ['item_name_category_tfidf_bigram_32', 'item_name_category_frequent_32', 'item_name_category_tfidf_bigram_256', 'item_name_category_tfidf_unigram_256', 'item_name_category_frequent_256', 'item_name_category_tfidf_unigram_32'], 'target_lagged.csv': ['target'], 'features_category.csv': ['item_category_id']}
read columns: ['item_name_category_tfidf_bigram_256_enc_train'
 'item_name_category_frequent_32_enc_train'
 'item_name_category_tfidf_unigram_256_enc_train' 'shop_id'
 'date_block_num' 'item_id'
 'item_name_category_tfidf_unigram_32_enc_train'
 'item_name_category_frequent_256_enc_train' 'item_cate

In [42]:
lists=[['target_lag_12', 'target_lag_13', 'target_lag_14', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']]
lists+=[['target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']]
lists+=[['target_lag_2', 'target_lag_3', 'target_lag_6']]
lists+=[['target_lag_2', 'target_lag_3', 'target_lag_4']]
lists+=[['target_lag_12',  'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']]
lists+=[['target_lag_12',  'target_lag_2', 'target_lag_3', 'target_lag_4']]

evaluate(' ',lists)

file -> columns mapping:  {'target_lagged.csv': ['target_lag_12', 'target_lag_4', 'target_lag_5', 'target', 'target_lag_3', 'target_lag_14', 'target_lag_13', 'target_lag_6', 'target_lag_2']}
read columns: ['target_lag_12' 'shop_id' 'target_lag_4' 'target_lag_5' 'date_block_num'
 'item_id' 'target' 'target_lag_3' 'target_lag_14' 'target_lag_13'
 'target_lag_6' 'target_lag_2']
(4499858,)
['target_lag_12', 'target_lag_13', 'target_lag_14', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']: Clipped RMSE 0.37270642671600185
['target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']: Clipped RMSE 0.37348156144421235
['target_lag_2', 'target_lag_3', 'target_lag_6']: Clipped RMSE 0.3773256767539508
['target_lag_2', 'target_lag_3', 'target_lag_4']: Clipped RMSE 0.37879492747742627
['target_lag_12', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']: Clipped RMSE 0.37257159613227697
['target_lag_12', 'target_lag_2',

In [43]:
lists=[['target_lag_12', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6']]
lists+=[['target_shop_lag_12', 'target_shop_lag_2', 'target_shop_lag_3', 'target_shop_lag_4', 'target_shop_lag_5', 'target_shop_lag_6']]
lists+=[['target_item_lag_12', 'target_item_lag_2', 'target_item_lag_3', 'target_item_lag_4', 'target_item_lag_5', 'target_item_lag_6']]
lists+=[['target_category_lag_12', 'target_category_lag_2', 'target_category_lag_3', 'target_category_lag_4', 'target_category_lag_5', 'target_category_lag_6']]

evaluate(' ',lists)

file -> columns mapping:  {'target_lagged.csv': ['target_lag_4', 'target_lag_6', 'target_lag_12', 'target_lag_5', 'target_lag_3', 'target', 'target_lag_2'], 'target_category_lagged.csv': ['target_category_lag_12', 'target_category_lag_4', 'target_category_lag_2', 'target_category_lag_6', 'target_category_lag_3', 'target_category_lag_5'], 'target_shop_lagged.csv': ['target_shop_lag_4', 'target_shop_lag_5', 'target_shop_lag_12', 'target_shop_lag_2', 'target_shop_lag_3', 'target_shop_lag_6'], 'target_item_lagged.csv': ['target_item_lag_4', 'target_item_lag_6', 'target_item_lag_12', 'target_item_lag_3', 'target_item_lag_2', 'target_item_lag_5']}
read columns: ['target_lag_12' 'shop_id' 'target_lag_4' 'target_lag_5' 'date_block_num'
 'item_id' 'target' 'target_lag_3' 'target_lag_6' 'target_lag_2'
 'target_category_lag_6' 'target_category_lag_4' 'target_category_lag_12'
 'target_category_lag_3' 'target_category_lag_2' 'target_category_lag_5'
 'target_shop_lag_2' 'target_shop_lag_3' 'target_s

In [44]:
lists=[['target_lag_12', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6','target_shop_lag_12', 'target_shop_lag_2', 'target_shop_lag_3', 'target_shop_lag_4', 'target_shop_lag_5', 'target_shop_lag_6','target_item_lag_12', 'target_item_lag_2', 'target_item_lag_3', 'target_item_lag_4', 'target_item_lag_5', 'target_item_lag_6']]

evaluate(' ',lists)

file -> columns mapping:  {'target_lagged.csv': ['target_lag_4', 'target_lag_6', 'target_lag_12', 'target_lag_5', 'target_lag_3', 'target', 'target_lag_2'], 'target_shop_lagged.csv': ['target_shop_lag_4', 'target_shop_lag_5', 'target_shop_lag_12', 'target_shop_lag_2', 'target_shop_lag_3', 'target_shop_lag_6'], 'target_item_lagged.csv': ['target_item_lag_4', 'target_item_lag_6', 'target_item_lag_12', 'target_item_lag_3', 'target_item_lag_2', 'target_item_lag_5']}
read columns: ['target_lag_12' 'shop_id' 'target_lag_4' 'target_lag_5' 'date_block_num'
 'item_id' 'target' 'target_lag_3' 'target_lag_6' 'target_lag_2'
 'target_shop_lag_2' 'target_shop_lag_3' 'target_shop_lag_4'
 'target_shop_lag_6' 'target_shop_lag_5' 'target_shop_lag_12'
 'target_item_lag_3' 'target_item_lag_2' 'target_item_lag_5'
 'target_item_lag_4' 'target_item_lag_6' 'target_item_lag_12']
(4499858,)
['target_lag_12', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6', 'target_shop_lag_12', 't